In [ ]:
# Import Dependencies
import hvplot.pandas
import holoviews as hv
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

In [ ]:
%%capture --no-display

vacation_map_plot = city_data_df.hvplot.points("Lng", "Lat", geo = True, color = "City",
                                          alpha = 0.75, size = "Humidity", tiles = "OSM")

# Display the map
vacation_map_plot

In [ ]:
# Narrow down cities that fit criteria and drop any results with null values
# Vacation Spot Criteria: a max temp lower than 90, higher than 80, 
# cloudiness less than 2, wind speed less than 20 mph

city_data_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 90)
                                         & (city_data_df["Max Temp"] > 80) 
                                         & (city_data_df["Cloudiness"] < 2)
                                         & (city_data_df["Wind Speed"] < 20)
                                        ]


# Drop any rows with null values
city_data_filtered_df.dropna()

# Display sample data
city_data_filtered_df

In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

In [ ]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
            
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel was found!"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

In [ ]:
%%capture --no-display

# remove rows for which no hotel was found because these shouldn't display in the map
hotel_df = hotel_df[hotel_df["Hotel Name"] != "No hotel was found!"]

# Configure the map plot
city_hotel_map_plot = hotel_df.hvplot.points("Lng", 
                                             "Lat", 
                                             geo = True,
                                             color = "City",
                                             alpha = 0.8,
                                             width = 600,
                                             height = 400,
                                             size = "Humidity",
                                             tiles = "OSM",
                                             hover_cols = ["Hotel Name", "Country"],
                                            )

# save the map to an external file
hv.save(city_hotel_map_plot, "output_data/City_Hotel_Map.html")

# Display the map
city_hotel_map_plot